# Find significant features
Compute a p value for each feature

Author: Andrei Todor

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind, f_oneway
from statsmodels.sandbox.stats.multicomp import multipletests

# Output format option
Short output will include mz, rt and pvals per line. Long output will also include all sample intensity values for each feature. Use short output for mummichog; long output for heat maps.

In [2]:
# include intensity values
long_output = 1

# Read inputs

In [3]:
# read feature table
infile = "../results/features_avg_pos.txt"
filter_data = pd.read_table(infile)
filter_header = [x for x in filter_data]

# set up sample groups
sample_files = ["../data/samples_p.csv", "../data/samples_mock.csv", "../data/samples_yf.csv"]
g = []
for gf in sample_files:
    f = open(gf, 'r')
    ids = [id.rstrip() for id in f.readlines() if len(id.strip()) > 0]
    if len(ids) > 0:
        g.append(ids)
    f.close()

# Statistical tests
If 2 classes, perform t test, otherwise ANOVA

Compute p-value and corrected p-value for each feature

Write output to file

In [4]:
pvals = []
for idx in range(filter_data.shape[0]):
    line = filter_data.iloc[idx]
    nl = ""
    data = []
    ndata = []
    for k in range(len(g)):
        data.append([line[i] for i in range(len(line)) if filter_header[i] in g[k]])
        ndata.append(len([x for x in data[k]]))
    if (all(x > 0 for x in ndata)):
        #should do a paired test, but we eliminated a sample from timepoint 0
        if len(g) == 2:
            (f, p) = ttest_ind(*data)
        else:
            (f, p) = f_oneway(*data)
        s = ""
        for k in range(len(g)):
            for j in range(len(g[k])):
                if g[k][j] in filter_header:
                    i = filter_header.index(g[k][j])
                    s = s + "\t" + str(line[i])
        pvals.append([idx, line[0], line[1], p, f, s])
bh = multipletests([x[3] for x in pvals], method = 'fdr_bh')

new_header = "mz\trt\tpval\tscore\tFDR"
if long_output == 1:
    for k in range(len(g)):
        for j in range(len(g[k])):
            if g[k][j] in filter_header:
                new_header = new_header + "\t" + g[k][j]

                
# write output
out_dir = "../results"
file = open(out_dir + "/pvals_pos.txt", "w")
file.write(new_header + "\n")
for i in range(len(pvals)):
    line = filter_data.iloc[pvals[i][0]]
    if long_output == 0:
        pvals[i][5] = ""
    file.write(str(line[0]) + "\t" + str(line[1]) + "\t" + str(pvals[i][3]) + "\t" + str(pvals[i][4]) + "\t" + str(bh[1][i]) + pvals[i][5] + "\n")